In [1]:
import torch
model = torch.hub.load("pytorch/vision", "vit_b_16")

import torchvision
from Models.transformer import VisionTransformer as vit
import Models.Conv as conv

from DataLoader import CIFAR100
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn

import utils

import numpy as np
import torch.backends.cudnn as cudnn
import random

Using cache found in /home/M2021080/.cache/torch/hub/pytorch_vision_main
/home/M2021080/.cache/torch/hub/pytorch_vision_main/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")
Using cache found in /home/M2021080/.cache/torch/hub/pytorch_vision_main
/home/M2021080/.cache/torch/hub/pytorch_vision_main/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
train_loader, test_loader = CIFAR100.get_data(32*3)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
depths = [11, 13, 16, 19]

criterion_onlylabel = lambda a,b : mse(a*b, b)
criterion_CE = nn.CrossEntropyLoss()
mse = nn.MSELoss()
softmax = torch.nn.Softmax(dim = 1)
criterion_KLD = torch.nn.KLDivLoss(reduction="batchmean")
criterion_response = lambda a,b : criterion_KLD(torch.log_softmax(a, dim=1),torch.softmax(b, dim=1))
CE_loss = nn.CrossEntropyLoss()


In [ ]:
for depth in depths:
    
    print(f"DEPTH : {depth}" + "!" * 200)
    torch.cuda.empty_cache()
    
    teacher = torch.hub.load("pytorch/vision", f'vgg{depth}_bn')
    teacher.classifier[-1] = nn.Linear(teacher.classifier[-1].in_features, 100)
    teacher.load_state_dict(torch.load(f"saved_models/vgg/vgg{depth}.pth").module.state_dict())
    teacher = conv.vgg_feature(100, depth, teacher)

    student = conv.vgg_feature(100, depth, pretrained = "IMAGENET1K_V1")


    device = "cuda"

    teacher = teacher.to(device)
    # teacher = torch.nn.DataParallel(teacher, device_ids=[0, 1, 2])

    student = student.to(device)
    # student = torch.nn.DataParallel(student, device_ids=[0, 1, 2])


    S_optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)
    T_optimizer = optim.SGD(teacher.parameters(), lr=0.01, momentum=0.9)

    S_scheduler = torch.optim.lr_scheduler.MultiStepLR(S_optimizer, milestones=[1,2,3,4,5,6,7], gamma=0.1)
    T_scheduler = torch.optim.lr_scheduler.MultiStepLR(T_optimizer, milestones=[1,2,3,4,5,6,7], gamma=0.1)

    utils.set_seed()
    best_acc = 0.0
    stack = 0

    accs_train = []
    accs_test = []

    utils.test(teacher, test_loader,device) # student도 변하는거 확인 완료함
    utils.test(student, test_loader,device) # student도 변하는거 확인 완료함

    length = student.length + 1

    torch.cuda.empty_cache()

    student_test_accs = []

    for epoch in range(100):

        print(f"lr : {S_scheduler.get_last_lr()}")

        if S_scheduler.get_last_lr()[0] < 0.000001:
            break

        T_correct = 0
        S_correct = 0
        all_data = 0

        loss_distill = []
        loss_CE = []
    #     loss_response = []
        student.train()
        teacher.eval()
        for img, label in tqdm(train_loader):
            input_data = img.to(device)
            label = label.to(device)


            all_data += len(input_data)
            input_lrp = utils.get_LRP_img(input_data, label, teacher, criterion_CE, T_optimizer, mean=1.5, std = 0.1, mult=0.4).cuda()

            S_optimizer.zero_grad()
            T_optimizer.zero_grad()

            layer = random.randint(0,  length)
            input_data, fk = student(input_data,layer)
            input_lrp, fk_lrp = teacher(input_lrp,layer)

            if layer < student.length:
                fk = torch.mean(fk, dim=1)
                fk_lrp = torch.mean(fk_lrp, dim=1)

            distill_loss = mse(fk, fk_lrp)
    #         distill_loss = (mse(torch.mean(fk, dim=2), torch.mean(fk_lrp, dim=2)) + mse(torch.mean(fk, dim=1), torch.mean(fk_lrp, dim=1))) / 2

            CE_loss = criterion_CE(input_data, label)

            response_loss = criterion_response(input_data, input_lrp)


            T_correct += sum(label == torch.argmax(input_lrp, dim=1))
            S_correct += sum(label == torch.argmax(input_data, dim=1))

            loss_CE.append(CE_loss.item())
            loss_distill.append(distill_loss.item())
    #         loss_response.append(response_loss.item())

            loss = (distill_loss * 2 + CE_loss) / 2
            loss.backward()
            S_optimizer.step()

        print("distill loss : ", sum(loss_distill) / len(loss_distill))
        print("general loss : ", sum(loss_CE) / len(loss_CE))
    #     print("response loss : ", sum(loss_response) / len(loss_response))

        print(f"Teacher acc: {T_correct / all_data}")
        print(f"Student acc: {S_correct / all_data}")

        test_acc = utils.test(student, test_loader,device, epoch) # student도 변하는거 확인 완료함

        if test_acc > best_acc + 0.01:
            stack = 0
            best_acc = test_acc

        else:
            stack+=1

        if stack > 3:
            S_scheduler.step()
            stack = 0

        student_test_accs.append(test_acc.item())
        print("=" * 100)


    saved_folder = "saved_models/vgg/"
    torch.save(student, f"{saved_folder}/vgg{depth}_student.pth")

    with open(f"{saved_folder}/vgg{depth}_student.json", "w") as f:
        json.dump({"student_test_accs" : student_test_accs}, f)
        
    del teacher
    del student

DEPTH : 11!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


Using cache found in /home/M2021080/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/M2021080/.cache/torch/hub/pytorch_vision_main
100%|██████████| 105/105 [00:17<00:00,  5.89it/s]


0 	 test acc : 0.7490999698638916


100%|██████████| 105/105 [00:17<00:00,  6.08it/s]


0 	 test acc : 0.014099999330937862
lr : [0.01]


...

100%|██████████| 521/521 [10:16<00:00,  1.18s/it]


distill loss :  0.1185673466706058
general loss :  1.940304742207225
Teacher acc: 0.9617199897766113
Student acc: 0.48597997426986694


100%|██████████| 105/105 [00:17<00:00,  6.09it/s]


1 	 test acc : 0.6660000085830688
lr : [0.01]


 70%|███████   | 366/521 [07:13<03:07,  1.21s/it]